In [1]:
import sys
from io import BytesIO

import base64
import requests
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from cartoonize_network.Transformer import Transformer

from tqdm import tqdm
from tqdm import tqdm_notebook

In [2]:
model = Transformer()
model.eval()
model.load_state_dict(torch.load("./cartoonize_pretrained_models/Hosoda_net_G_float.pth"))

<All keys matched successfully>

In [3]:
def transform(model, input_image):
    model.cuda()

    h, w = input_image.size

    input_image = input_image.resize((h, w), Image.BICUBIC)
    input_image = np.asarray(input_image)

    input_image = input_image[:, :, [2, 1, 0]]
    input_image = transforms.ToTensor()(input_image).unsqueeze(0)

    input_image = -1 + 2 * input_image
    input_image = Variable(input_image).cuda()

    with torch.no_grad():
        output_image = model(input_image)[0]

    output_image = output_image[[2, 1, 0], :, :]
    output_image = output_image.data.cpu().float() * 0.5 + 0.5

    output_image = output_image.numpy()

    output_image = np.uint8(output_image.transpose(1, 2, 0) * 255)
    output_image = Image.fromarray(output_image)

    return output_image

In [4]:
ffhq_folder_path = "./uncartoonized"
cartoonized_folder_path = "./cartoonized"

for file in tqdm(os.listdir(ffhq_folder_path), desc="Processing images"):
    if file.endswith(".png"):
        image_path = os.path.join(ffhq_folder_path, file)
        with Image.open(image_path) as img:
            output = transform(model, img)
            output_path = os.path.join(cartoonized_folder_path, file)  # 保留原始圖片名稱
            output.save(output_path)

Processing images: 100%|███████████████████████████████████████████████████████████| 5000/5000 [09:48<00:00,  9.20it/s]
